In [7]:
import nltk
import pandas as pd
import numpy as np
import scipy
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from scipy.sparse import hstack
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

data = pd.read_pickle("training_data_pct1.pkl")
relative = data[["KeyFile","TableFormOrder"]].sort_values(['KeyFile', 'TableFormOrder'], ascending=[True, True])
relative.reset_index(inplace=True)
relative.drop("index", axis=1, inplace=True)
relative["relativeIndex"] = '0'

currentKeyFile = ""
currentRelativeIndex = 0
for i in range(len(relative)):
    if (relative.iloc[i,0] != currentKeyFile):
        currentKeyFile = relative.iloc[i,0]
        currentRelativeIndex = 0
    relative.iloc[i,2] = (currentRelativeIndex)
    currentRelativeIndex +=1


relative.head()


,KeyFile,TableFormOrder,relativeIndex
0,1733007,1000,0
1,1733007,2000,1
2,1733007,3000,2
3,1733007,4000,3
4,1733007,5000,4


In [2]:
import tensorflow as tf

C:\Users\bryan_tran\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
from tensorflow.contrib.learn.python.learn.estimators import DNNClassifier

In [8]:
data.head()

,KeyFile,KeyExtractedDocumentTable,TableFormOrder,ExtractedDocumentTable,KeyExtractedTableNames,label_name,label,value_text_concat,set
0,24860523,2f03ae0d-3f38-4dd8-8493-0000fe323ef8,6000,consolidated statements of cash flows,3.0,Cash Flow,2.0,cash flows from operating activities: net loss...,train
1,38184177,43341021-0e8a-4342-8c6b-00017ffc5eb6,23000,consolidated statements of operations,1.0,Income Statement,1.0,revenues: circulation revenues: circulation re...,train
2,11317061,0bb4cb2a-354a-4e00-a2ba-0005234967ee,33000,impairment of receivables from customers,9999.0,N/A,249.0,real-estate financing real-estate financing co...,train
3,390786062,d17432ea-913f-40d0-9ae5-00112d837455,1000,capital adequacy,172.0,Capital Adequacy,10.0,credit risk (standardized approach) institutes...,train
4,35703753,c6e92fa2-0762-47de-a196-000a12cdd61d,4000,asset/liability maturity analysis - of,267.0,Asset/Liability Maturity Analysis - OF,21.0,total assets: 30 days total assets: 60 days to...,train


In [13]:
relative[(relative["KeyFile"] == 24860523.0) & (relative["TableFormOrder"] == 6000)]

,KeyFile,TableFormOrder,relativeIndex
23726,24860523,6000,2


In [14]:
from nltk.corpus import stopwords

def text_process(mess):

    nopunc = [char for char in mess if char not in string.punctuation]

    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [103]:
# extracted["value_text_concat"] = extracted['value_text_concat'].apply(text_process)

# extracted['value_text_concat'] = extracted.apply(lambda x: text_process(x["value_text_concat"]))


%time extracted['value_text_concat'] = extracted.apply(lambda x: text_process(x['value_text_concat']), axis = 1)

Wall time: 1h 36min 22s


C:\Users\bryan_tran\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [104]:
%time extracted['ExtractedDocumentTable'] = extracted.apply(lambda x: text_process(x['ExtractedDocumentTable']), axis = 1)

Wall time: 1min 28s


C:\Users\bryan_tran\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# " ".join(extracted["value_text_concat"][0])

In [124]:
%time extracted['value_text_concat'] = extracted.apply(lambda x: " ".join(x["value_text_concat"]), axis = 1)

Wall time: 4 s


C:\Users\bryan_tran\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
%time extracted['ExtractedDocumentTable'] = extracted.apply(lambda x: " ".join(x["ExtractedDocumentTable"]), axis = 1)

In [132]:
%time extracted['newText'] = extracted.apply(lambda x: x["ExtractedDocumentTable"]+ " "+x["value_text_concat"], axis = 1)

Wall time: 2.31 s


C:\Users\bryan_tran\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
extracted = pd.read_csv("extracted.csv")
extracted.dropna(inplace=True)
data.reset_index(inplace=True)
extracted["KeyFile"] = data["KeyFile"]
extracted.drop("Unnamed: 0", axis = 1, inplace=True)


extracted2 = pd.merge(extracted, relative,  how='left', left_on=['KeyFile','TableFormOrder'], right_on = ['KeyFile','TableFormOrder'])
extracted = extracted2
extracted.head()

extracted = extracted.join(pd.get_dummies(extracted["relativeIndex"], drop_first=True))

In [16]:
extracted.shape

(31804, 178)

In [94]:
extracted["label_name"]  = extracted["label_name"].fillna("Not appropriate")

In [141]:
# extracted = extracted.join(pd.get_dummies(extracted["TableFormOrder"], drop_first=True))

In [29]:
extracted["label_name"].nunique()

246

In [17]:
extracted.head()

,TableFormOrder,ExtractedDocumentTable,label_name,value_text_concat,newText,KeyFile,relativeIndex,1,2,3,...,171,173,174,175,176,179,182,183,184,228
0,6000,consolidated statements cash flows,Cash Flow,cash flows operating activities net loss cash ...,consolidated statements cash flows cash flows ...,24860523,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,23000,consolidated statements operations,Income Statement,revenues circulation revenues circulation reve...,consolidated statements operations revenues ci...,38184177,21,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000,capital adequacy,Capital Adequacy,credit risk standardized approach institutes c...,capital adequacy credit risk standardized appr...,390786062,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4000,assetliability maturity analysis,Asset/Liability Maturity Analysis - OF,total assets 30 days total assets 60 days tota...,assetliability maturity analysis total assets ...,35703753,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4000,condensed consolidated statements operations,Income Statement,revenues investment banking revenue revenues i...,condensed consolidated statements operations r...,14299187,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
msg_train, msg_test, label_train, label_test = \
train_test_split(extracted.drop(["TableFormOrder","ExtractedDocumentTable","label_name","value_text_concat","KeyFile","relativeIndex"], axis =1), extracted['label_name'], test_size=0.2)
bow_transformer = CountVectorizer(max_features=1000).fit(msg_train['newText'])


In [42]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]
classifier = DNNClassifier(feature_columns=feature_columns,hidden_units=[25443000,20,10], n_classes=246)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000002881127240>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\BRYAN_~1\\AppData\\Local\\Temp\\tmprorzzhuz'}


In [43]:
messages_bow = bow_transformer.transform(msg_train['newText'])
tfidf_transformer = TfidfTransformer().fit(messages_bow)
messages_tfidf = tfidf_transformer.transform(messages_bow)

X_train_dtm = messages_tfidf

# messages_bow = bow_transformer.transform(msg_test['newText'])
# messages_tfidf = tfidf_transformer.transform(messages_bow)
# d1 = msg_test.iloc[:,1:]
# d1 = scipy.sparse.csr_matrix(d1.values)
# X_train_dtm = hstack((messages_tfidf,d1))


# all_predictions = testmodel.predict(X_train_dtm)


classifier.fit(X_train_dtm, label_train)

ValueError: Labels dtype should be integer Instead got <dtype: 'string'>.

In [41]:
X_train_dtm

<25443x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 798840 stored elements in Compressed Sparse Row format>

In [18]:
# messages_bow = bow_transformer.transform(msg_train['newText'])
# tfidf_transformer = TfidfTransformer().fit(messages_bow)
# messages_tfidf = tfidf_transformer.transform(messages_bow)
# d1 = msg_train.iloc[:,1:]
# d1 =scipy.sparse.csr_matrix(d1.values)
# X_train_dtm = hstack((messages_tfidf,d1))

# from sklearn.model_selection import GridSearchCV
# param_grid = { 
#     'n_estimators': [30,100,110],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     "min_samples_split": [15, 20],
#     "min_samples_leaf": [5, 10, 20],
#     "criterion": ['gini', 'entropy']
# }


# CV_rfc = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv= 5)
# CV_rfc.fit(X_train_dtm, label_train)

In [37]:
messages_bow = bow_transformer.transform(msg_train['newText'])
tfidf_transformer = TfidfTransformer().fit(messages_bow)
messages_tfidf = tfidf_transformer.transform(messages_bow)
d1 = msg_train.iloc[:,1:]
d1 =scipy.sparse.csr_matrix(d1.values)
X_train_dtm = hstack((messages_tfidf,d1))
#  %time testmodel = GradientBoostingClassifier(learning_rate=0.01, n_estimators=100,verbose=1).fit(X_train_dtm, label_train)
# testmodel = RandomForestClassifier(n_estimators=100,verbose=1,n_jobs=2).fit(X_train_dtm, label_train)
testmodel = LinearSVC().fit(X_train_dtm, label_train)

messages_bow = bow_transformer.transform(msg_test['newText'])
messages_tfidf = tfidf_transformer.transform(messages_bow)
d1 = msg_test.iloc[:,1:]
d1 = scipy.sparse.csr_matrix(d1.values)
X_train_dtm = hstack((messages_tfidf,d1))


all_predictions = testmodel.predict(X_train_dtm)


In [38]:
print (classification_report(label_test, all_predictions))

                                                                         precision    recall  f1-score   support

                                        5 Classifications of Loans - OF       0.25      0.12      0.17         8
                                                              AOCI - OF       0.50      0.14      0.22         7
                                                      Access Lines - OF       0.50      1.00      0.67         1
                             Accounts Payable and Accrued Expenses - OF       0.80      0.89      0.84         9
                                  Accumulated Comprehensive Income - OF       0.62      0.50      0.55        36
                                         Allowance for Loan Losses - OF       0.35      0.25      0.29        32
                                             Appendix 5B Cash Flow - OF       0.94      1.00      0.97        16
                                              Asset Maturity Table - OF       0.32      0.26   

C:\Users\bryan_tran\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\bryan_tran\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [47]:
accuracy_score(label_test, all_predictions)

0.7167112089294136

In [44]:
def predict_table(clf,text_features,eval_param,N_tables=10):
   
    if eval_param==0:
        return(pd.DataFrame({"class_label":clf.predict(text_features).tolist()}))
    
    if eval_param==1:
        dfres=pd.DataFrame({"pred_prob":clf.predict_proba(text_features)[0].tolist(),\
                            "class_label":clf.classes_.tolist()}).sort_values("pred_prob",ascending=False)
        return(dfres.iloc[:N_tables])
    
    if eval_param==2:
        dfres=pd.DataFrame({"pred_prob":clf.predict_proba(text_features)[0].tolist(),\
                            "class_label":clf.classes_.tolist()}).sort_values("pred_prob",ascending=False)
        return(dfres)
    

In [9]:
dftable

(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
             oob_score=False, random_state=None, verbose=1,
             warm_start=False),
 <6361x5171 sparse matrix of type '<class 'numpy.float64'>'
 	with 223342 stored elements in COOrdinate format>,
 1)

In [45]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]
pipeline = Pipeline([
('bow', CountVectorizer()),
    ('tfidf', TfidfTransformer()), 
#     ('classifier', RandomForestClassifier(n_estimators=100,verbose=1,n_jobs=-1)),  
    ('classifier', DNNClassifier(feature_columns=feature_columns,hidden_units=[25443000,20,10], n_classes=246)),
])

msg_train, msg_test, label_train, label_test = \
train_test_split(extracted['newText'], extracted['label_name'], test_size=0.2)


%time pipeline.fit(msg_train,label_train)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000028814B19E8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\BRYAN_~1\\AppData\\Local\\Temp\\tmpocp0v1ys'}


ValueError: Labels dtype should be integer Instead got <dtype: 'string'>.

In [59]:

predictions = pipeline.predict(msg_test)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.8s finished


In [8]:
print (classification_report(predictions, label_test))

NameError: name 'predictions' is not defined

In [61]:
accuracy_score(label_test, predictions)

0.7582141172771577

In [27]:
df = extracted.copy()

In [24]:
docs=extracted['newText'][2]

In [28]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

%time extracted['newText'] = extracted.apply(lambda x: lemmatize_text(x['newText']), axis = 1)


Wall time: 1min 49s


In [32]:
%time extracted['newText'] = extracted.apply(lambda x: " ".join(x["newText"]), axis = 1)

Wall time: 25.2 s
